# Spectra

## Imports

In [2]:
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from mpl_toolkits.axes_grid1 import ImageGrid

plt.rc('text', usetex=True)
plt.rc('font', size=48)

## Constants

In [4]:
v_th = -12.8

## Parameters

In [5]:
# Thetas

num_thetas = 9
cos_thetas = np.linspace(0,1,num_thetas+1) 
cos_thetas_plus = np.linspace(-1,0,num_thetas+1) 

thetas = np.sort(np.array([np.arccos(ct) for ct in cos_thetas]))
thetas_plus = np.array([np.arccos(ctp) for ctp in cos_thetas_plus])

print(np.rad2deg(thetas))
print(np.rad2deg(thetas_plus))

[  0.          27.26604445  38.94244127  48.1896851   56.2510114
  63.61220004  70.52877937  77.16041159  83.62062979  90.        ]
[ 180.          152.73395555  141.05755873  131.8103149   123.7489886
  116.38779996  109.47122063  102.83958841   96.37937021   90.        ]


In [6]:
# Selected thetas: 0, 45, 90

thetas_selected = [0, 45, 90]
thetas_selected_inds = [0, 2, 8] # normal: 0-1, 2-3, 8-9   plus: 1-0, 3-2, 9-8
thetas_linestyles = ['-', '-.', ':']

## Functions

In [7]:
def read_data(vrot, vout, logtau):
    
    df_directory = '../../data/tau10E'+str(logtau)+'/vrot'+str(vrot)+'/vout'+str(vout)+'/'
    df_fname = 'tau10E'+str(logtau)+'_vrot'+str(vrot)+'_vout'+str(vout)+'_out.ascii'
    df = pd.read_csv(df_directory+df_fname, delimiter=' ')
    
    escaped = df['escaped']
    x_frec = df['x_frec']
    z_u = df['z_u']

    inds = np.where(escaped == 0)[0]

    x_frec_escaped = np.array(x_frec)[inds]
    z_u_escaped = np.array(z_u)[inds]

    return x_frec_escaped, z_u_escaped


def get_histogram(x):
    
    n, b = np.histogram(x, bins=40)        
    delta_x = b[1]-b[0]
    area = delta_x * sum(n)
    n = n/area

    border_x = []
    border_y = []

    for j in range(len(n)-1):
        border_x.append(b[j]+delta_x/2)
        border_x.append(b[j]+delta_x/2)
        border_y.append(n[j])
        border_y.append(n[j+1])
        
    #return b[0:40], n, border_x, border_y
    return border_x, border_y

In [14]:
def make_grid_plot_voutfixed(logtaus, vrots, vout):
    
    num_rows, num_cols = len(logtaus), len(vrots)
    
    fig = plt.figure(1, figsize=(30, 20))
    grid = ImageGrid(fig, 111, # similar to subplot(111)
                     nrows_ncols = (num_rows, num_cols), # creates num_rows x num_cols grid of axes
                     axes_pad=0.3, # pad between axes in inch.
                     aspect=False,)
    props = dict(boxstyle='square', facecolor='white')
    
    i = 0 # grid cell number
    for logtau in logtaus:
        for vrot in vrots:

            x_frec, z_u = read_data(vrot, vout, logtau)
            acos_z_u = np.array([np.arccos(z) for z in z_u])
            
            grid_i = grid[i]
            
            for t in range(len(thetas_selected)):
                
                ts = thetas_selected[t]
                tsi = thetas_selected_inds[t]
                tl = thetas_linestyles[t]
                                
                theta_low = thetas[tsi]
                theta_up = thetas[tsi+1]
                theta_plus_low = thetas_plus[tsi+1]
                theta_plus_up = thetas_plus[tsi]
                
                #only the ones between those upper and lower angles
                angle_indices = np.where( ((acos_z_u >= theta_low) & (acos_z_u < theta_up)) | 
                                          ((acos_z_u >= theta_plus_low) & (acos_z_u < theta_plus_up)) )[0]
                
                x = x_frec[angle_indices] * v_th
                
                border_x, border_y = get_histogram(x)
                
                theta_lab = r'$\theta=$ ' + '${0:.0f}$'.format(ts) + '$^\circ$'
                
                if i == 6:
                    grid_i.plot(border_x, border_y, c='k', linewidth=1, linestyle=tl, label=theta_lab)
                    grid_i.legend(loc='upper right')
                else:
                    grid_i.plot(border_x, border_y, c='k', linewidth=1, linestyle=tl)
            
            grid_i.axvline(x=0, ymin=0, ymax=1, c='k', linestyle='--', linewidth=1)
            
            xmin, xmax = -250,750
            ymin, ymax = 0, 0.012
            grid_i.set_xlim(xmin,xmax)
            grid_i.set_ylim(ymin,ymax)
            
            logtau_lab = r'$\tau=10$ ' + '$^{0:.0f}$'.format(logtau) 
            vrot_lab = r'$v_{\mathrm{rot}}=$ ' + '${0:.0f}$'.format(vrot) + ' $\mathrm{km}$ $\mathrm{s^{-1}}$'
            
            if i <= 2:
                grid_i.text((xmin+xmax)/2, ymax*1.2, vrot_lab, bbox=props,
                            horizontalalignment='center', verticalalignment='center')
            if i%3 == 2:
                grid_i.text(xmax*1.2, (ymin+ymax)/2, logtau_lab, bbox=props, rotation=270,
                            horizontalalignment='center', verticalalignment='center')
            if i == 3:
                grid_i.set_ylabel('$\mathrm{Intensity}$')
            if i == 7:
                grid_i.set_xlabel('$\mathrm{V}$ ($\mathrm{km}$ $\mathrm{s^{-1}}$)')
            
            i += 1

    plt.savefig('./results/varying_rotation.png', format='png', transparent=False)#, bbox_inches='tight')
    plt.close()
    
    
def make_grid_plot_vrotfixed(logtaus, vrot, vouts):
    
    num_rows, num_cols = len(logtaus), len(vouts)
    
    fig = plt.figure(1, figsize=(30, 20))
    grid = ImageGrid(fig, 111, # similar to subplot(111)
                     nrows_ncols = (num_rows, num_cols), # creates num_rows x num_cols grid of axes
                     axes_pad=0.3, # pad between axes in inch.
                     aspect=False,)
    props = dict(boxstyle='square', facecolor='white')
    
    i = 0 # grid cell number
    for logtau in logtaus:
        for vout in vouts:

            x_frec, z_u = read_data(vrot, vout, logtau)
            acos_z_u = np.array([np.arccos(z) for z in z_u])
            
            grid_i = grid[i]
            
            for t in range(len(thetas_selected)):
                
                ts = thetas_selected[t]
                tsi = thetas_selected_inds[t]
                tl = thetas_linestyles[t]
                                
                theta_low = thetas[tsi]
                theta_up = thetas[tsi+1]
                theta_plus_low = thetas_plus[tsi+1]
                theta_plus_up = thetas_plus[tsi]
                
                #only the ones between those upper and lower angles
                angle_indices = np.where( ((acos_z_u >= theta_low) & (acos_z_u < theta_up)) | 
                                          ((acos_z_u >= theta_plus_low) & (acos_z_u < theta_plus_up)) )[0]
                
                x = x_frec[angle_indices] * v_th
                
                border_x, border_y = get_histogram(x)
                
                theta_lab = r'$\theta=$ ' + '${0:.0f}$'.format(ts) + '$^\circ$'
                
                if i == 6:
                    grid_i.plot(border_x, border_y, c='k', linewidth=1, linestyle=tl, label=theta_lab)
                    grid_i.legend(loc='upper right')
                else:
                    grid_i.plot(border_x, border_y, c='k', linewidth=1, linestyle=tl)
                
            grid_i.axvline(x=0, ymin=0, ymax=1, c='k', linestyle='--', linewidth=1)
            
            #grid_i.set_xticks([-200,0,200,400,600], [-200,0,200,400,600])
            
            xmin, xmax = -300,750
            ymin, ymax = 0, 0.0125
            grid_i.set_xlim(xmin,xmax)
            grid_i.set_ylim(ymin,ymax)
            
            logtau_lab = r'$\tau=10$ ' + '$^{0:.0f}$'.format(logtau) 
            vout_lab = r'$v_{\mathrm{out}}=$ ' + '${0:.0f}$'.format(vout) + ' $\mathrm{km}$ $\mathrm{s^{-1}}$'
            
            if i <= 2:
                grid_i.text((xmin+xmax)/2, ymax*1.2, vout_lab, bbox=props,
                            horizontalalignment='center', verticalalignment='center')
            if i%3 == 2:
                grid_i.text(xmax*1.2, (ymin+ymax)/2, logtau_lab, bbox=props, rotation=270,
                            horizontalalignment='center', verticalalignment='center')
            if i == 3:
                grid_i.set_ylabel('$\mathrm{Intensity}$')
            if i == 7:
                grid_i.set_xlabel('$\mathrm{V}$ ($\mathrm{km}$ $\mathrm{s^{-1}}$)')
            
            i += 1

    plt.savefig('./results/varying_outflow.png', format='png', transparent=False)#, bbox_inches=4)
    plt.close()

# Figure: varying_rotation

In [15]:
logtaus = [5,6,7]
vrots = [0,50,100]
vout = 50

make_grid_plot_voutfixed(logtaus, vrots, vout)

# Figure: varying_outflow

In [16]:
logtaus = [5,6,7]
vrot = 50
vouts = [25,50,75]

make_grid_plot_vrotfixed(logtaus, vrot, vouts)